In [1]:
import pandas as pd
import numpy as np

X_train = pd.read_csv('./cvss_2022_2024_X_train.csv')
y_train = pd.read_csv('./cvss_2022_2024_y_train.csv')

X_test = pd.read_csv('./cvss_2022_2024_X_test.csv')
y_test = pd.read_csv('./cvss_2022_2024_y_test.csv')

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

print(y_train['cvssv3_confidentiality_impact'].value_counts(dropna=False) / y_train.shape[0])
print(y_test['cvssv3_confidentiality_impact'].value_counts(dropna=False) / y_test.shape[0])
print(X_train.info())
print(y_train.info())

(80769, 1)
(20193, 1)
(80769, 1)
(20193, 1)
cvssv3_confidentiality_impact
NONE    0.333990
HIGH    0.333383
LOW     0.332628
Name: count, dtype: float64
cvssv3_confidentiality_impact
LOW     0.336156
HIGH    0.333135
NONE    0.330709
Name: count, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80769 entries, 0 to 80768
Data columns (total 1 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   english_description  80769 non-null  object
dtypes: object(1)
memory usage: 631.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80769 entries, 0 to 80768
Data columns (total 1 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   cvssv3_confidentiality_impact  80769 non-null  object
dtypes: object(1)
memory usage: 631.1+ KB
None


In [2]:
label_column_name = "cvssv3_confidentiality_impact"
train_labels = y_train.loc[:, label_column_name]
test_labels = y_test.loc[:, label_column_name]


from sklearn.preprocessing import LabelEncoder
import pickle

le = LabelEncoder()
le.fit(train_labels)
print(le.classes_)

with open("../labels/confidentiality_impact_label.txt", "wb") as f:
    pickle.dump(le.classes_, f)

NUM_CLASSES = len(le.classes_)
print(NUM_CLASSES)

encoded_train_labels = le.transform(train_labels)
encoded_test_labels = le.transform(test_labels)

print(train_labels[:10], encoded_train_labels[:10])
print(len(X_train), len(train_labels), len(X_test), len(test_labels))

['HIGH' 'LOW' 'NONE']
3
0     LOW
1     LOW
2    HIGH
3    HIGH
4    NONE
5     LOW
6     LOW
7     LOW
8    HIGH
9    NONE
Name: cvssv3_confidentiality_impact, dtype: object [1 1 0 0 2 1 1 1 0 2]
80769 80769 20193 20193


In [3]:
X_train.loc[:,"english_description"].tolist()

['"An issue in animal-art-lab v13.6.1 allows attackers to send crafted notifications via leakage of the channel access token."',
 '"Unauth. Reflected Cross-Site Scripting (XSS) vulnerability in Poll Maker Team Poll Maker plugin <=\xa04.7.0 versions."',
 '"Cross-Site Request Forgery (CSRF) vulnerability in Jens Törnell WP Page Numbers plugin <=\xa00.5 versions."',
 '"An issue discovered in casdoor v1.636.0 allows attackers to obtain sensitive information via the ssh.InsecureIgnoreHostKey() method."',
 '"Vulnerability of spoofing trustlists of Huawei desktop.Successful exploitation of this vulnerability can cause third-party apps to hide app icons on the desktop to prevent them from being uninstalled."',
 '"Zoho ManageEngine ADSelfService Plus before 6121 allows XSS via the welcome name attribute to the Reset Password, Unlock Account, or User Must Change Password screen."',
 '"Multiple vulnerabilities in the web-based management interface of Cisco Security Manager could allow an unauthen

In [4]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('prajjwal1/bert-small')

d:\my-proj\ML - Proje\CVSS-Prediction-BERT_small\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
train_encodings = tokenizer(X_train.loc[:,"english_description"].tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test.loc[:,"english_description"].tolist(), truncation=True, padding=True, max_length=128)

In [6]:
import torch

class CVEDataset(torch.utils.data.Dataset):
    def __init__(self, X, encodings, labels, encoded_labels):
        self.texts = X.loc[:,"english_description"].tolist()
        self.encodings = encodings
        self.labels = labels.tolist()
        self.encoded_labels = encoded_labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['text_labels'] = self.labels[idx]
        item['encoded_labels'] = torch.tensor(self.encoded_labels[idx])
        item['vulnerability_description'] = self.texts[idx]
        
        return item

    def __len__(self):
        return len(self.labels)


In [7]:
train_dataset = CVEDataset(X_train, train_encodings, train_labels, encoded_train_labels)
test_dataset = CVEDataset(X_test, test_encodings, test_labels, encoded_test_labels)

In [8]:
train_dataset[0]

{'input_ids': tensor([  101,  1000,  2019,  3277,  1999,  4111,  1011,  2396,  1011,  6845,
          1058, 17134,  1012,  1020,  1012,  1015,  4473, 17857,  2000,  4604,
         19275, 26828,  2015,  3081, 17271,  4270,  1997,  1996,  3149,  3229,
         19204,  1012,  1000,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [9]:
train_dataset[0]['input_ids']

tensor([  101,  1000,  2019,  3277,  1999,  4111,  1011,  2396,  1011,  6845,
         1058, 17134,  1012,  1020,  1012,  1015,  4473, 17857,  2000,  4604,
        19275, 26828,  2015,  3081, 17271,  4270,  1997,  1996,  3149,  3229,
        19204,  1012,  1000,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [10]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=16)
next(iter(train_loader))

{'input_ids': tensor([[  101,  1000,  2019,  ...,     0,     0,     0],
         [  101,  1000, 14477,  ...,     0,     0,     0],
         [  101,  1000,  2892,  ...,     0,     0,     0],
         ...,
         [  101,  1000,  2023,  ...,     0,     0,     0],
         [  101,  1000, 24156,  ...,     0,     0,     0],
         [  101,  1000,  1996,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'text_labels': ['LOW',
  'LOW',
  'HIGH',
  'HIGH',
  'NONE',
  'LOW',
  'LOW',
  'LOW',
  'HIGH',
  'NONE',
  'LOW',
  'LOW',
 

In [11]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('prajjwal1/bert-small', num_labels=NUM_CLASSES)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-1

In [12]:
for param in model.base_model.parameters():
    param.requires_grad = False
    
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

classifier.weight tensor([[-0.0137, -0.0304,  0.0029,  ..., -0.0159, -0.0050, -0.0003],
        [-0.0099, -0.0011, -0.0061,  ...,  0.0004,  0.0079, -0.0321],
        [-0.0099, -0.0159,  0.0050,  ...,  0.0032, -0.0141,  0.0137]])
classifier.bias tensor([0., 0., 0.])


In [13]:
from torch.utils.data import DataLoader
from transformers import AdamW
import torch.nn.functional as F

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

training_loss_batch = []
training_loss_epoch = []

for epoch in range(3):
    model.train()
    training_loss = 0
    num_correct = 0 
    num_examples = 0
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['encoded_labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()
        training_loss_batch.append(loss.data.item())
        training_loss += loss.data.item() * input_ids.size(0)
        correct = torch.eq(torch.max(F.softmax(outputs.logits, dim=1), dim=1)[1], labels)
        num_correct += torch.sum(correct).item()
        num_examples += correct.shape[0]
    training_loss /= len(train_loader.dataset)
    training_loss_epoch.append(training_loss)
    
    print('Epoch: {}, Training Loss: {}, Training Accuracy = {}'.format(epoch, training_loss, num_correct / num_examples))

  

d:\my-proj\ML - Proje\CVSS-Prediction-BERT_small\venv\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 0, Training Loss: 0.9871138399392269, Training Accuracy = 0.5266376951553194
Epoch: 1, Training Loss: 0.8997727042446267, Training Accuracy = 0.5917616907476878
Epoch: 2, Training Loss: 0.8681404454002104, Training Accuracy = 0.6063712562988275


In [14]:
for param in model.base_model.parameters():
    param.requires_grad = True
    
for name, param in model.named_parameters():
    if param.requires_grad:
        # print(name, param.data)
        pass

In [15]:
from torch.utils.data import DataLoader
from transformers import AdamW
import torch.nn.functional as F

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

training_loss_batch = []
training_loss_epoch = []

for epoch in range(3):
    model.train()
    training_loss = 0
    num_correct = 0 
    num_examples = 0
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['encoded_labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()
        training_loss_batch.append(loss.data.item())
        training_loss += loss.data.item() * input_ids.size(0)
        correct = torch.eq(torch.max(F.softmax(outputs.logits, dim=1), dim=1)[1], labels)
        num_correct += torch.sum(correct).item()
        num_examples += correct.shape[0]
    training_loss /= len(train_loader.dataset)
    training_loss_epoch.append(training_loss)
    
    print('Epoch: {}, Training Loss: {}, Training Accuracy = {}'.format(epoch, training_loss, num_correct / num_examples))

  

Epoch: 0, Training Loss: 0.3607343608352863, Training Accuracy = 0.8567272096967896
Epoch: 1, Training Loss: 0.23971707709252796, Training Accuracy = 0.9104483155666159
Epoch: 2, Training Loss: 0.1733723337674384, Training Accuracy = 0.9379836323341876


In [16]:
model.save_pretrained('../models/bert-small-vulnerability_confidentiality_impact-classification')

In [17]:
test_loader = DataLoader(test_dataset, batch_size=16)

model.eval()
num_correct = 0 
num_examples = 0
test_loss = 0
for batch in test_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['encoded_labels'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs[0]
    test_loss += loss.data.item() * input_ids.size(0)
    correct = torch.eq(torch.max(F.softmax(outputs.logits, dim=1), dim=1)[1], labels)
    num_correct += torch.sum(correct).item()
    num_examples += correct.shape[0]
test_loss /= len(test_loader.dataset)

        
print('Test Loss: {}, Test Accuracy = {}'.format(test_loss, num_correct / num_examples))

Test Loss: 0.22315704322694932, Test Accuracy = 0.9251720893378894
